```1.``` Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам в словаре применяется только одна операция – удаление символа (1-n). Чтобы найти исправление из слова тоже удаляются символы и сравниваются с теми, что хранятся в словаре. Оцените качество полученного алгоритма теми же тремя метриками.


https://medium.com/@wolfgarbe/1000x-faster-spelling-correction-algorithm-2012-8701fcd87a5f

In [54]:
from collections import Counter, defaultdict
from nltk import sent_tokenize
import gzip
import csv
import pandas as pd
import re
from operator import itemgetter
import nltk

In [2]:
data = pd.read_csv('lenta-ru-news.csv')

In [3]:
texts = data['text']

In [4]:
def align_words(sent_1, sent_2):
    tokens_1, tokens_2 = normalize(sent_1), normalize(sent_2)
    return zip(tokens_1, tokens_2)

In [6]:
def normalize(text):
    pat = re.compile(r'[А-Яа-я]+')
    normalized = re.findall(pat, text)

    return [word.lower() for word in normalized if word]

In [7]:
corpus = []
for text in texts[:12000]:
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [8]:
vocab = set()

for sent in corpus:
    vocab.update(sent)

In [9]:
words = Counter()

for sent in corpus:
    words.update(sent)

In [10]:
word_freq = dict(words)

for key, value in word_freq.items():
    word_freq[key] = value/len(words)

In [11]:
def n_deletes_helper(word, min_len=0):
    result = set()
    # taking word length into account
    if len(word) > min_len:
        for i in range(len(word)):
            result.add(word[:i] + word[i+1:])
    else:
        result.add(word)
    return result

In [12]:
# нужно сохранять предыдущие удаления!

In [13]:
def n_deletes(word, edit_distance=2, min_len=2):
    edit_distance -= 1
    forms = list(n_deletes_helper(word, min_len))
    tmp = set(forms)
    while edit_distance > 0:
        edit_distance -= 1
        new_forms = []
        for form in tmp:
            new_forms.extend(n_deletes_helper(form, min_len))
        tmp = set(new_forms)
        forms.extend(tmp)
        
    return forms

In [14]:
n_deletes('word', edit_distance=2)

['wor', 'ord', 'wod', 'wrd', 'wr', 'od', 'wo', 'wd', 'rd', 'or']

In [15]:
sym_vocab = {}
for word in vocab:
    sym_vocab[word] = word
    forms = n_deletes(word)
    for form in forms:
        if form not in sym_vocab.keys():
            sym_vocab[form] = word
        else:
            if isinstance(sym_vocab[form], set):
                sym_vocab[form].add(word)
            else:
                sym_vocab[form] = {sym_vocab[form]}
                sym_vocab[form].add(word)

```
There are four different comparison pair types:
dictionary entry==input entry,
delete(dictionary entry,p1)==input entry
dictionary entry==delete(input entry,p2)
delete(dictionary entry,p1)==delete(input entry,p2)
```

In [16]:
for word in sym_vocab['односоронни']:
    print(word_freq[word], word)

7.693787266782073e-06 односторонний
3.077514906712829e-05 односторонние
3.077514906712829e-05 односторонних
1.5387574533564147e-05 односторонним


In [17]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
good = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [18]:
def most_probable(words, word_freq=word_freq):
    freqs = dict()
    for word in words:
        freqs[word] = word_freq[word]
    return max(freqs.items(), key=itemgetter(1))[0]

In [19]:
def vocab_helper(word, vocab=vocab):
    return word in vocab

In [20]:
'вообще' in vocab

True

In [21]:
sym_vocab['вбще'], sym_vocab['вобще']

('вообще', 'вообще')

In [22]:
def correct(word, sym_vocab=sym_vocab):
    if vocab_helper(word):
        return word
    # we don't expect to reliably spellcheck two- or one-letter words
    elif len(word) < 3:
        return word
    else:
        if word in sym_vocab.keys(): 
            candidates = sym_vocab[word]
            if isinstance(candidates, set):
                return most_probable(candidates)
            else:
                return candidates
        else:
            forms = n_deletes(word)
            candidates = []
            for form in forms:
                if form in sym_vocab.keys():
                    cand = sym_vocab[form]
                    if isinstance(cand, set):
                        candidates.extend(cand)
                    else:
                        candidates.append(cand)
            if candidates:
                return most_probable(candidates)
            else:
                # the word is expected to have errors in it,
                # but we don't have a form to replace it with
                return f'*{word}'

In [29]:
good_corrections_list = []

def good_corrections(correction, corrections_list=good_corrections_list):
    if correction[0] != correction[1]:
        corrections_list.append(correction)

In [30]:
def correction_helper(correction, good_prediction_count, bad_prediction_count, error_count, overall, 
                      keep_good_corrections=True):
    append = False
    # if we predicted correctly
    if correction[0] == correction[2]:
        good_prediction_count += 1
        if keep_good_corrections:
            good_corrections(correction)
    # otherwise
    else:
        bad_prediction_count += 1
        append = True
    
    # actual errors
    if correction[0] != correction[1]:
        error_count += 1
    
    # total pairs checked
    overall += 1
    
    return good_prediction_count, bad_prediction_count, error_count, overall, append

In [31]:
cache = dict()
corrections = []
good_preds = 0
bad_preds = 0
errors = 0
total = 0

for i in range(len(good)):
    append = False
    pairs = align_words(good[i], bad[i])
    for pair in pairs:
        left = pair[0]
        right = pair[1]
        if right in cache.keys():
            correction = left, right, cache[right]
        else:
            corrected_word = correct(right)
            correction = left, right, corrected_word
            cache[right] = corrected_word


        good_preds, bad_preds, errors, total, append = correction_helper(correction,
                                                                         good_preds,
                                                                         bad_preds,
                                                                         errors,
                                                                         total)
        if append:
            corrections.append(correction)

    

В целом работает не так уж плохо, хотя слова делить всё равно не умеет.

In [32]:
good_corrections_list[:10]

[('апофеозом', 'опофеозом', 'апофеозом'),
 ('отсутствие', 'отсуствие', 'отсутствие'),
 ('основная', 'основая', 'основная'),
 ('напрасно', 'нарасно', 'напрасно'),
 ('сегодняшнее', 'сегодяшнее', 'сегодняшнее'),
 ('потому', 'патаму', 'потому'),
 ('лучше', 'лчше', 'лучше'),
 ('компьютерная', 'компютерная', 'компьютерная'),
 ('что', 'чтото', 'что'),
 ('участвовать', 'учавствовать', 'участвовать')]

In [33]:
Counter(corrections).most_common(10)

[(('вообще', 'ваще', 'все'), 16),
 (('кстати', 'кстате', 'сайте'), 15),
 (('очень', 'оооочень', '*оооочень'), 9),
 (('это', 'ето', 'место'), 9),
 (('насчет', 'нащет', 'нет'), 6),
 (('здесь', 'сдесь', 'есть'), 6),
 (('можно', 'можна', 'она'), 5),
 (('девчонки', 'девченки', 'девочки'), 5),
 (('что', 'што', 'шуток'), 5),
 (('очень', 'ооооочень', '*ооооочень'), 5)]

```
Для оценки используем будем использовать три метрики:
1) процент правильных слов;
2) процент исправленных ошибок
3) процент ошибочно исправленных правильных слов
```

In [48]:
good_preds/total # верно предсказанные слова (1)

0.802691924227318

In [49]:
len(good_corrections_list)/errors # сколько ошибок исправленно верно (2)

0.3029932803909591

In [50]:
bad_preds/(total - errors) # ошибочно исправленные слова (3)

0.23579173120457525

In [42]:
errors/total # всего ошибок в  исходном тексте

0.16321036889332005

```2.``` Добавьте к полученному алгоритму исправления триграммную модель и проверьте, улучшает ли она качество.

In [55]:
tri_model = defaultdict(lambda: defaultdict(lambda: 0))
for sentence in corpus:
    for w1, w2, w3 in nltk.trigrams(sentence, pad_right=True, pad_left=True, left_pad_symbol='<s>', right_pad_symbol='</s>'):
        tri_model[(w1, w2)][w3] += 1